If you are interested in graident boosting, here is a good place to start: https://xgboost.readthedocs.io/en/latest/tutorials/model.html

This is a supervised machine learning method.

XGB does not like to have multiple outputs: https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html

In [ ]:
!pip install catboost
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.1)


In [ ]:
# If you have installation questions, please reach out

import seaborn as sns
import pandas as pd # data storage
import catboost as cats # graident boosting 

from catboost import CatBoostRegressor, Pool

import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

print('catboost ver:', cats.__version__)
print('scikit ver:', sklearn.__version__)

import datetime

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import median_absolute_error, max_error, mean_squared_error

catboost ver: 0.24.4
scikit ver: 0.24.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading in Entire Dataset

In [ ]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')

df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df.describe()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

In [ ]:
df.columns.values

array(['depth_ft', 'USGS_ID', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 'PE_smooth', 'GR_smooth',
       'gz_linear_interp', 'gz_pchip_interp', 'perm_air_md',
       'perm_klink_md', 'He_por', 'horz_perm_md', 'effective_por',
       'por_percent', 'stress_swanson_perm_md', 'sample', 'Na', 'Na Err',
       'Mg', 'Mg Err', 'Al', 'Al Err', 'Si', 'Si Err', 'P', 'P Err', 'S',
       'S Err', 'K', 'K Err', 'Ca', 'Ca Err', 'Ti', 'Ti Err', 'Cr',
       'Cr Err', 'Mn', 'Mn Err', 'Fe', 'Fe Err', 'Co', 'Co Err', 'Ni',
       'Ni Err', 'Cu', 'Cu Err', 'Zn', 'Zn Err', 'Ba', 'Ba Err'],
      dtype=object)

In [ ]:
dataset = df[[
    'depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'Ti', 'Mg', 'Al', 'Ca', 'Si'
]]

In the next code block, we will remove the rows without data, and change string NaN's to np.nans

In [ ]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
dataset = dataset.dropna()
np.shape(dataset)

dataset.head(3)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,Ti,Mg,Al,Ca,Si
0,7741.0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,0.1641,0.6987,2.8101,4.5640,33.9921
5,7741.5,0.626112,0.182393,0.063209,0.586778,0.825487,0.351442,0.922670,0.001810,0.001810,0.196369,0.357995,0.1746,0.9547,2.9962,3.8975,33.9915
10,7742.0,0.621729,0.184446,0.063426,0.590484,0.825878,0.402086,0.886733,0.001889,0.001889,0.211611,0.360458,0.1555,0.7553,2.5388,4.1047,33.3257


In [ ]:
X = dataset[['depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

Y2 = dataset[['Ti', 'Mg', 'Al', 'Ca', 'Si']]


In [ ]:
#scaler = MinMaxScaler()
#scaler = RobustScaler()

#Y = scaler.fit_transform(Y2[Y2.columns].to_numpy())
#df_Y = pd.DataFrame(data=Y2, columns=Y2.columns)

## Starting to set up the ML model params

In [ ]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y2.values, test_size=test_size)

### Let's try some hyperparameter tuning (this takes forever!)

Hyperparameter testing does a grid search to find the best parameters, out of the parameters below. This turned out to be really slow on my laptop. Please skip this! 

In [ ]:
model = CatBoostRegressor(objective='MultiRMSE', iterations=2000)

model.fit(X_train, y_train, verbose=500 )

preds = model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print("Root Mean Squared Error: %f" % (rmse))
#max = max_error(y_test, preds)
#print("Max Error: %f" % (max))

MAE = median_absolute_error(y_test, preds)
print("Median Abs Error: %f" % (MAE))

0:	learn: 5.4331445	total: 14.1ms	remaining: 28.1s
500:	learn: 2.6786930	total: 6.82s	remaining: 20.4s
1000:	learn: 2.2038084	total: 13.6s	remaining: 13.6s
1500:	learn: 1.8970152	total: 20.4s	remaining: 6.79s
1999:	learn: 1.6675165	total: 27.3s	remaining: 0us
Root Mean Squared Error: 1.067390
Median Abs Error: 0.446644


In [ ]:
grid = {'learning_rate': [0.05, 0.1, 0.2, 0.3], # 0.1
        'depth': [ 4, 6, 8], #6
        'l2_leaf_reg': [3, 4, 5, 6]}

model_grid = CatBoostRegressor(objective='MultiRMSE', iterations=2000)

# Grid Search
grid_search_result = model_grid.grid_search(grid, 
                                            X=X_train, 
                                            y=y_train, 
                                            cv=5,
                                            verbose=False
                                       )


bestTest = 3.428386896
bestIteration = 1915


bestTest = 3.457030287
bestIteration = 402


bestTest = 3.435169151
bestIteration = 273


bestTest = 3.534126275
bestIteration = 266


bestTest = 3.408326311
bestIteration = 1903


bestTest = 3.402008163
bestIteration = 1845


bestTest = 3.430761166
bestIteration = 453


bestTest = 3.504614219
bestIteration = 184


bestTest = 3.428291177
bestIteration = 1999


bestTest = 3.441318399
bestIteration = 1792


bestTest = 3.408458503
bestIteration = 407


bestTest = 3.526728634
bestIteration = 296


bestTest = 3.434576624
bestIteration = 1681


bestTest = 3.448795084
bestIteration = 1594


bestTest = 3.416799792
bestIteration = 506


bestTest = 3.458463687
bestIteration = 211


bestTest = 3.312948168
bestIteration = 1633


bestTest = 3.393898482
bestIteration = 606


bestTest = 3.36541955
bestIteration = 302


bestTest = 3.374055251
bestIteration = 424


bestTest = 3.32192702
bestIteration = 1858


bestTest = 3.355040029
bestIteration = 950


be

In [ ]:
grid_search_result['params']

{'depth': 6, 'l2_leaf_reg': 3, 'learning_rate': 0.05}

In [ ]:
model2 = CatBoostRegressor(objective='MultiRMSE',
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           iterations=2000)

model2.fit(X_train, y_train, verbose=500 )

preds2 = model2.predict(X_test)

rmse2 = mean_squared_error(y_test, preds2, squared=False)
print("Root Mean Squared Error: %f" % (rmse2))

MAE2 = median_absolute_error(y_test, preds2)
print("Median Abs Error: %f" % (MAE2))

0:	learn: 5.3796778	total: 14.8ms	remaining: 29.5s
500:	learn: 2.3388525	total: 6.84s	remaining: 20.5s
1000:	learn: 1.8190859	total: 13.7s	remaining: 13.7s
1500:	learn: 1.4892705	total: 20.5s	remaining: 6.82s
1999:	learn: 1.2504720	total: 27.3s	remaining: 0us
Root Mean Squared Error: 1.060620
Median Abs Error: 0.424598


# Export

In [ ]:
x = datetime.datetime.now()

d = {'target': ['MultiXRF', 'MultiXRF'],
     'MSE': [rmse, rmse2],
     'MAE': [MAE, MAE2],
     'day': [x.day, x.day], 
     'month':[x.month, x.month], 
     'year':[x.year, x.year],
     'model':['catboost', 'catboost'],
     'version':[cats.__version__, cats.__version__ ]}

results = pd.DataFrame(data=d)

results.to_csv('drive/My Drive/1_lewis_research/analysis/experiments/catboost/catboost_results/multi_xrf_cat.csv')

results

,target,MSE,MAE,day,month,year,model,version
0,MultiXRF,1.06739,0.446644,16,2,2021,catboost,0.24.4
1,MultiXRF,1.06062,0.424598,16,2,2021,catboost,0.24.4


# Need to fix below!

In [ ]:
true_preds = scaler.inverse_transform(preds)
true_preds_df = pd.DataFrame(data = true_preds,   
                  columns = ['Ba', 'Na', 'Mg','Si', 'Al', 'K', 'Cu', 'Fe'])
 
y_test_trans = scaler.inverse_transform(y_test)
y_test_df = pd.DataFrame(data = y_test_trans,   
                  columns = ['Ba', 'Na', 'Mg','Si', 'Al', 'K', 'Cu', 'Fe'])

NameError: ignored

In [ ]:
true_preds_df.describe(percentiles=[0.05, 0.95])

In [ ]:
yoyo = pd.DataFrame(data = preds,   
                  columns = ['Ba', 'Na', 'Mg','Si', 'Al', 'K', 'Cu', 'Fe'])

yoyo.describe(percentiles=[0.05, 0.95])

In [ ]:
y_test_df.describe(percentiles=[0.05, 0.95])

In [ ]:
df80 = pd.DataFrame(data = y_test,   
                  columns = ['Ba', 'Na', 'Mg','Si', 'Al', 'K', 'Cu', 'Fe'])

In [ ]:
df80.describe()

In [ ]:
a= (preds-y_test)

In [ ]:
df100 = pd.DataFrame(data = a,   
                  columns = ['Ba', 'Na', 'Mg','Si', 'Al', 'K', 'Cu']) 

In [ ]:
df100.describe(percentiles=[.20,.80])

Now plug in the hyperparameters into the training model.

In [ ]:
error = preds - y_test

In [ ]:
error[0]

In [ ]:
np.shape(error[0])

In [ ]:
plt.figure(figsize=(6,3))
plt.hist(error[0], bins=25)
plt.xlabel('Prediction Error XRF')
plt.xlim((-4,4))